### Setup AWS Credentials

In [ ]:
import os
# Setup your AWS Access Key and Secret Key as environment variables.
os.environ["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] 

In [4]:
# Setup Nova Model
NOVA_MODEL_ID = "us.amazon.nova-pro-v1:0"

### Dataset Adapter

Initialize the Dataset Adapter that takes the input_columns and output_columns. We use the CSVDatasetAdapter to read a `.csv` file and adapt it to the standardized format. We also use the adapter to create train and test sets for our use case.

In [19]:
from amzn_nova_prompt_optimizer.core.input_adapters.dataset_adapter import CSVDatasetAdapter

input_columns = {"input"}
output_columns = {"answer"}

dataset_adapter = CSVDatasetAdapter(input_columns, output_columns)

# Adapt
dataset_adapter.adapt("../data/FacilitySupportAnalyzer.csv")

train_set, test_set = dataset_adapter.split(0.5)

### Prompt Adapter

Initialize the Prompt Adapter for the Original Prompt. For this example, we use the FacilitySupportAnalyzer System and User Prompt in the `.txt` format. 

In [20]:
from amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter import TextPromptAdapter

prompt_variables = input_columns

prompt_adapter = TextPromptAdapter()

prompt_adapter.set_system_prompt(file_path="original_prompt/system_prompt.txt", variables=prompt_variables)
prompt_adapter.set_user_prompt(file_path="original_prompt/user_prompt.txt", variables=prompt_variables)

# Adapt
prompt_adapter.adapt()

### Metric Adapter

Initialize the Metric Adapter for evaluating this prompt for certain optimizers. For this example, we build a Custom Metric for the FacilitySupportAnalyzer Dataset. The metric adapter requires the use of the `apply` [For single row evaluation] or `batch_apply` [For evaluating the whole dataset together] function

In [21]:
from amzn_nova_prompt_optimizer.core.input_adapters.metric_adapter import MetricAdapter
from typing import List, Any, Dict
import re
import json

class FacilitySupportAnalyzerMetric(MetricAdapter):
    def parse_json(self, input_string: str):
        """
        Attempts to parse the given string as JSON. If direct parsing fails,
        it tries to extract a JSON snippet from code blocks formatted as:
            ```json
            ... JSON content ...
            ```
        or any code block delimited by triple backticks and then parses that content.
        """
        try:
            return json.loads(input_string)
        except json.JSONDecodeError as err:
            error = err

        patterns = [
            re.compile(r"```json\s*(.*?)\s*```", re.DOTALL | re.IGNORECASE),
            re.compile(r"```(.*?)```", re.DOTALL)
        ]

        for pattern in patterns:
            match = pattern.search(input_string)
            if match:
                json_candidate = match.group(1).strip()
                try:
                    return json.loads(json_candidate)
                except json.JSONDecodeError:
                    continue

        raise error

    def _calculate_metrics(self, y_pred: Any, y_true: Any) -> Dict:
        strict_json = False
        result = {
            "is_valid_json": False,
            "correct_categories": 0.0,
            "correct_sentiment": False,
            "correct_urgency": False,
        }

        try:
            y_true = y_true if isinstance(y_true, dict) else (json.loads(y_true) if strict_json else self.parse_json(y_true))
            y_pred = y_pred if isinstance(y_pred, dict) else (json.loads(y_pred) if strict_json else self.parse_json(y_pred))
        except json.JSONDecodeError:
            result["total"] = 0
            return result  # Return result with is_valid_json = False
        else:
            if isinstance(y_pred, str):
                result["total"] = 0
                return result  # Return result with is_valid_json = False
            result["is_valid_json"] = True

            categories_true = y_true.get("categories", {})
            categories_pred = y_pred.get("categories", {})

            if isinstance(categories_true, dict) and isinstance(categories_pred, dict):
                correct = sum(
                    categories_true.get(k, False) == categories_pred.get(k, False)
                    for k in categories_true
                )
                result["correct_categories"] = correct / len(categories_true) if categories_true else 0.0
            else:
                result["correct_categories"] = 0.0  # or raise an error if you prefer

            result["correct_sentiment"] = y_pred.get("sentiment", "") == y_true.get("sentiment", "")
            result["correct_urgency"] = y_pred.get("urgency", "") == y_true.get("urgency", "")

        # Compute overall metric score
        result["total"] = sum(
            float(result[k]) for k in ["correct_categories", "correct_sentiment", "correct_urgency"]
        ) / 3.0

        return result

    def apply(self, y_pred: Any, y_true: Any):
        return self._calculate_metrics(y_pred, y_true)

    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        evals = [self.apply(y_pred, y_true) for y_pred, y_true in zip(y_preds, y_trues)]
        float_keys = [k for k, v in evals[0].items() if isinstance(v, (int, float, bool))]
        return {k: sum(e[k] for e in evals) / len(evals) for k in float_keys}

metric_adapter = FacilitySupportAnalyzerMetric()

### Inference Adapter
Initialize the InferenceAdapter to choose the backend Inference. Currently, we only support BedrockInferenceAdapter.

In [22]:
from amzn_nova_prompt_optimizer.core.inference.adapter import BedrockInferenceAdapter

inference_adapter = BedrockInferenceAdapter(region_name="us-east-1")

### Evaluator

The Evaluator can use the metric_adapter, prompt_adapter, and dataset_adapter to evaluate the prompt given the `model_id` to produce an evaluation score. The Evaluator internally uses the `InferenceRunner` to first generate inference results and then evaluate the output.

#### Base Model Evaluation

In [23]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

evaluator = Evaluator(prompt_adapter, test_set, metric_adapter, inference_adapter)

In [24]:
original_prompt_score = evaluator.aggregate_score(model_id=NOVA_MODEL_ID)

print(f"Original Prompt Evaluation Score = {original_prompt_score}")

2025/07/02 20:46:37 INFO amzn_nova_prompt_optimizer.core.evaluation: Cache miss - Running new inference on Dataset
Running inference: 100%|██████████████████████████████████████████████████████████████████████████| 100/100 [00:48<00:00,  2.05it/s]
2025/07/02 20:47:25 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Batch Evaluation on Dataset, using `batch_apply` metric
2025/07/02 20:47:25 INFO amzn_nova_prompt_optimizer.core.evaluation: Using cached inference results
2025/07/02 20:47:25 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Evaluation on Dataset, using `apply` metric


Original Prompt Evaluation Score = {'is_valid_json': 0.31, 'correct_categories': 0.281, 'correct_sentiment': 0.2, 'correct_urgency': 0.26, 'total': 0.247}


### Optimization Adapter

We can now define the Optimization Functions. The Optimization function takes as input the Prompt Adapter and Optionally a Dataset Adapter, Inference Adapter, and Metric Adapter. The optimization function optimizes the prompt and returns a Prompt Adapter.

In [9]:
class FacilitySupportAnalyzerNovaMetric(FacilitySupportAnalyzerMetric):
    def apply(self, y_pred: Any, y_true: Any):
        # Requires to return a value and not a JSON payload
        return self._calculate_metrics(y_pred, y_true)["total"]
        
    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        pass
nova_metric_adapter = FacilitySupportAnalyzerNovaMetric()

#### NovaPromptOptimizer

NovaPromptOptimizer = Nova Meta Prompter + MIPROv2 with Nova Model Tips

In [10]:
from amzn_nova_prompt_optimizer.core.optimizers import NovaPromptOptimizer

nova_prompt_optimizer = NovaPromptOptimizer(prompt_adapter=prompt_adapter, inference_adapter=inference_adapter, dataset_adapter=train_set, metric_adapter=nova_metric_adapter)

optimized_prompt_adapter = nova_prompt_optimizer.optimize(mode="pro")

2025/07/02 20:18:53 INFO amzn_nova_prompt_optimizer.core.optimizers.nova_meta_prompter.nova_mp_optimizer: Optimizing prompt using Nova Meta Prompter
2025/07/02 20:18:59 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using us.amazon.nova-pro-v1:0 for Evaluation
2025/07/02 20:18:59 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using us.amazon.nova-premier-v1:0 for Prompting
2025/07/02 20:18:59 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.custom_adapters.custom_chat_adapter: Initializing CustomChatAdapter with enable_json_fallback=False
2025/07/02 20:18:59 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using Nova tips for MIPROv2 optimization
2025/07/02 20:18:59 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/02 20:18:59 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instruc

Bootstrapping set 1/20
Bootstrapping set 2/20
Bootstrapping set 3/20


  8%|███████▋                                                                                        | 4/50 [00:06<01:10,  1.54s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/20


  8%|███████▋                                                                                        | 4/50 [00:05<00:58,  1.28s/it]


...

2025/07/02 20:36:26 INFO dspy.evaluate.evaluate: Average Metric: 25.8 / 35 (73.7%)
2025/07/02 20:36:26 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 73.71 on minibatch of size 35 with parameters ['Predictor 0: Instruction 8', 'Predictor 0: Few-Shot Set 11'].
2025/07/02 20:36:26 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [72.38, 68.67, 71.81, 67.71, 65.14, 70.48, 72.48, 66.38, 71.81, 71.9, 69.81, 69.62, 64.86, 65.62, 74.76, 70.48, 71.14, 76.76, 71.62, 73.14, 70.19, 75.14, 69.62, 75.05, 69.52, 63.9, 69.9, 74.1, 73.71]
2025/07/02 20:36:26 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.4, 73.2, 71.07, 73.67, 74.0, 72.4]
2025/07/02 20:36:26 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 74.0
2025/07/02 20:36:26 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/02 20:36:26 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 36 / 37 - Minibatch ==



Average Metric: 22.57 / 35 (64.5%): 100%|███████████████████████████████████████████████████████████| 35/35 [00:27<00:00,  1.27it/s]

2025/07/02 20:36:53 INFO dspy.evaluate.evaluate: Average Metric: 22.566666666666666 / 35 (64.5%)
2025/07/02 20:36:53 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 64.48 on minibatch of size 35 with parameters ['Predictor 0: Instruction 13', 'Predictor 0: Few-Shot Set 8'].
2025/07/02 20:36:53 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [72.38, 68.67, 71.81, 67.71, 65.14, 70.48, 72.48, 66.38, 71.81, 71.9, 69.81, 69.62, 64.86, 65.62, 74.76, 70.48, 71.14, 76.76, 71.62, 73.14, 70.19, 75.14, 69.62, 75.05, 69.52, 63.9, 69.9, 74.1, 73.71, 64.48]
2025/07/02 20:36:53 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.4, 73.2, 71.07, 73.67, 74.0, 72.4]
2025/07/02 20:36:53 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 74.0
2025/07/02 20:36:53 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/02 20:36:53 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 37 / 37 - Full Evaluation =====
2025/0


Average Metric: 36.47 / 50 (72.9%): 100%|███████████████████████████████████████████████████████████| 50/50 [00:13<00:00,  3.84it/s]

2025/07/02 20:37:06 INFO dspy.evaluate.evaluate: Average Metric: 36.46666666666667 / 50 (72.9%)
2025/07/02 20:37:06 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [70.4, 73.2, 71.07, 73.67, 74.0, 72.4, 72.93]
2025/07/02 20:37:06 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 74.0
2025/07/02 20:37:06 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/02 20:37:06 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/02 20:37:06 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 74.0!


In [11]:
optimized_prompt_adapter.show()

2025/07/02 20:37:07 INFO amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter: 
Standardized Prompt:


{
  "user_prompt": {
    "variables": [
      "input"
    ],
    "template": "Extract and return a json with the following keys and values from the input: [{{input}}]\n- \"urgency\" as one of `high`, `medium`, `low`\n- \"sentiment\" as one of `negative`, `neutral`, `positive`\n- \"categories\" as a dictionary with boolean values for: `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`, `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`, `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`, `facility_management_issues`\n\nYour complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnecessary whitespaces.",
    "metadata": {
      "format": "text"
    }
  },
  "system_prompt": {
    "variables": [],
    "template": "Analyze the custo

### Optimized System Prompt

In [12]:
print(optimized_prompt_adapter.system_prompt)

Analyze the customer service email to classify its content accurately. Return a JSON object with "urgency" (high/medium/low), "sentiment" (negative/neutral/positive), and "categories" (boolean flags for predefined types). Ensure compliance with data privacy standards by omitting personal identifiers. Follow the format strictly: `{"urgency": "...", "sentiment": "...", "categories": {"...": true/false, ...}}`.


### Optimized User Prompt

In [13]:
print(optimized_prompt_adapter.user_prompt)

Extract and return a json with the following keys and values from the input: [{{input}}]
- "urgency" as one of `high`, `medium`, `low`
- "sentiment" as one of `negative`, `neutral`, `positive`
- "categories" as a dictionary with boolean values for: `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`, `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`, `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`, `facility_management_issues`

Your complete message should be a valid json string that can be read directly and only contain the keys mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnecessary whitespaces.


### Few Shot Examples

In [14]:
print(f"Number of Few-Shot Examples = {len(optimized_prompt_adapter.few_shot_examples)}")

Number of Few-Shot Examples = 4


In [15]:
# Print only the first example
print(optimized_prompt_adapter.few_shot_examples[0])

{'input': 'Extract and return a json with the following keys and values from the input: [Subject: Urgent Request for Training and Support\n\nHi ProCare Support Team,\n\nI hope this message finds you well. My name is [Sender], and I manage a residential complex that has been utilizing ProCare Facility Solutions for our maintenance and cleaning needs for the past year. Your services have been instrumental in keeping our environment safe and well-maintained.\n\nHowever, I am reaching out with an urgent request. We are in immediate need of comprehensive training for our in-house maintenance team. Given the complexity of our facility\'s systems, particularly the HVAC and electrical components, it\'s crucial that our team is well-versed in best practices and troubleshooting techniques. \n\nWe have encountered several issues recently that have highlighted gaps in our current knowledge and capabilities. While we have managed to address these problems temporarily, a more sustainable solution is

### Evaluator

Now we evaluate the Nova Prompt Optimizer Optimized prompt

In [16]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

evaluator = Evaluator(optimized_prompt_adapter, test_set, metric_adapter, inference_adapter)

In [17]:
nova_prompt_optimizer_evaluation_score = evaluator.aggregate_score(model_id=NOVA_MODEL_ID)
print(f"Nova Prompt Optimizer = {nova_prompt_optimizer_evaluation_score}")

2025/07/02 20:40:54 INFO amzn_nova_prompt_optimizer.core.evaluation: Cache miss - Running new inference on Dataset
Running inference: 100%|██████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.18it/s]
2025/07/02 20:41:40 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Batch Evaluation on Dataset, using `batch_apply` metric
2025/07/02 20:41:40 INFO amzn_nova_prompt_optimizer.core.evaluation: Using cached inference results
2025/07/02 20:41:40 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Evaluation on Dataset, using `apply` metric


Nova Prompt Optimizer = {'is_valid_json': 1.0, 'correct_categories': 0.95, 'correct_sentiment': 0.65, 'correct_urgency': 0.77, 'total': 0.79}


In [18]:
optimized_prompt_adapter.save("optimized_prompt/")